In [ ]:
!pip install requests #Instalacion de los request

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import csv
import re
import time
import random

In [2]:
# corrector de longitud de fila
def lenCorrector(rowInfo):
  evalList = ["own_pistol_tt","own_pistol_ct","own_score_tt","rival_score_tt","own_score_ct","rival_score_ct"]
  for col in evalList:
    if len(rowInfo["map"])> len(rowInfo[col]):
      rowInfo[col].append(0)
  if len(rowInfo["map"])> len(rowInfo["team"]):
    rowInfo["team"].append(rowInfo["team"][len(rowInfo["team"])-1])
    rowInfo["rival"].append(rowInfo["rival"][len(rowInfo["rival"])-1])


In [3]:
def picksBansFiltrado(picksBans):
  """Function for separete the picks this proccess the pickBans, bans and decider by team, need optimization
  in the amount of list, probably a dict is a better option """

  teams = [] #teams[0] always team A, teams[1] always team B
  banA = [] #bans for the team A
  banB = [] #bans for the team B
  pickA = [] #picks team A
  pickB = [] #picks team B
  decider = [] #decider map
  for count, listaPickBan in enumerate(picksBans):
    listaTemp = listaPickBan.split()
    if count == 0:
      teams.append(listaTemp[0])
    if count == 1:
      teams.append(listaTemp[0])
  for count, listaPickBan in enumerate(picksBans):
    listaTemp = listaPickBan.split()
    if listaTemp[1] == "ban":
      if listaTemp[0] == teams[0]:
        banA.append(listaTemp[2])
      if listaTemp[0] == teams[1]:
        banB.append(listaTemp[2])
    if listaTemp[1] == "pick":
      if listaTemp[0] == teams[0]:
        pickA.append(listaTemp[2])
      if listaTemp[0] == teams[1]:
        pickB.append(listaTemp[2])
    if listaTemp[1] == "remains":
      decider.append(listaTemp[0])
  return teams,banA,banB,pickA,pickB,decider

In [4]:
def picksBansCsv(teams,banA,banB,pickA,pickB,decider):
  """Create a list with the bans for team
  falta checkear si funciona para bo5, problema con el bo5 columnas del header"""
  teamA = []
  teamB = []
  header = ["team","ban1","ban2","le ban1","le ban2","pick1","decider","order"]
  teamA.append(teams[0])
  for banPropio in banA:
    teamA.append(banPropio)
  for banRival in banB:
    teamA.append(banRival)
  for pick in pickA:
    teamA.append(pick)
  for deci in decider:
    teamA.append(deci)
  teamA.append("A")
  teamB.append(teams[1])
  for banPropio in banB:
    teamB.append(banPropio)
  for banRival in banA:
    teamB.append(banRival)
  for pick in pickB:
    teamB.append(pick)
  for deci in decider:
    teamB.append(deci)
  teamB.append("B")

  return header,teamA,teamB

In [5]:
#Creating the file
def createCsvPicks(filename, head):
  with open(fileName,'a') as file:
    writer =csv.writer(file)
    writer.writerow(head)

def writeFunction(filename,teamA,teamB):
  with open(filename,'a') as file:
    writer = csv.writer(file)
    writer.writerow(teamA)
    writer.writerow(teamB)
    file.close()

def createCsvDict(filename,rows): #Creation of the file and headers
  with open(filename, "a") as csvfile: #creation of the file with te headers
    writer = csv.writer(csvfile)
    writer.writerow(rows.keys())
    csvfile.close()

def dictWritter(filename,rows): #stats from the teams
      with open(filename, 'a') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(zip(*rows.values()))
        csvfile.close()

In [7]:
def pistolRounds(soup,rowInfo,bo=3):
  if bo == 3:
    rondas = [x for x in range(1,67,13)]
  else:
    rondas = [x for x in range(1,119,13)]

  bloques_pistol = soup.find_all('div', class_="vlr-rounds-row-col")
  try:
    for ronda in rondas:
      rondaEvaluada = re.findall(r"rnd-sq(.*)",str(bloques_pistol[ronda]))
      if rondaEvaluada[0] == ' mod-win mod-ct">':
        rowInfo["own_pistol_ct"].append(1)
      elif rondaEvaluada[0] == ' mod-win mod-t">':
        rowInfo["own_pistol_tt"].append(1)
      elif rondaEvaluada[1] == ' mod-win mod-ct">':
        rowInfo["own_pistol_tt"].append(0)
      elif rondaEvaluada[1] == ' mod-win mod-t">':
        rowInfo["own_pistol_ct"].append(0)
      if rondaEvaluada[0] == "rnd-sq" and rondaEvaluada[1] == "rnd-sq":
        rowInfo['own_pistol_ct'].append(0)
        rowInfo['own_pistol_tt'].append(0)
  except:
    pass

def extraerScore(soup,mapasJuntos,rowInfo,bo=3):
  """
  Function for scores by side and map played
  what is missing: Map pick owner
  """

  bloques_equipos = soup.find_all('div', class_='team')
  for map in mapasJuntos:
    rowInfo["map"].append(map)
  for count, bloque in enumerate(bloques_equipos):
    if count%2 == 0:
      try:
        equipo = bloque.find("div",class_="team-name").text.strip()
        rowInfo["team"].append(equipo)
        spans = bloque.find_all("span")
        for span in spans:
          if span["class"][0] == "mod-t":
            rowInfo["own_score_tt"].append(span.text.strip())
          if span["class"][0] == "mod-ct":
            rowInfo["own_score_ct"].append(span.text.strip())
      except:
        pass
    if count%2 == 1:
      try:
        equipo = bloque.find("div",class_="team-name").text.strip()
        rowInfo["rival"].append(equipo)
        spans = bloque.find_all("span")
        for span in spans:
          if span["class"][0] == "mod-t":
            rowInfo["rival_score_tt"].append(span.text.strip())
          if span["class"][0] == "mod-ct":
            rowInfo["rival_score_ct"].append(span.text.strip())

      except:
        pass
  pistolRounds(soup,rowInfo,3)
  lenCorrector(rowInfo)
  return rowInfo

#Extract total scores from a match
def scoreTotal(soup):
  score = soup.find('div',class_='match-header-vs-score')
  spanScores = score.find_all("span")
  equipoIzq = int(spanScores[0].text.strip())
  equipoDer = int(spanScores[2].text.strip())
  return equipoIzq+equipoDer



In [8]:
def pickBans(soup):
  picksBans = soup.find("div",{"class":"match-header-note"}).get_text() #Extraccion del texto
  picksBans = picksBans.strip() #Remover los separados y espacios
  picksBans = picksBans.split(";")
  return picksBans

In [9]:
def runExtractor(soup,filename='PicksBans.csv'):
  pBans = pickBans(soup)
  teams,banA,banB,pickA,pickB,decider = picksBansFiltrado(pBans) #Picks & Bans filter
  header, teamA, teamB = picksBansCsv(teams,banA,banB,pickA,pickB,decider) #header, teamA and teamB
  mapasJuntos = [pickA[0],pickB[0],decider[0]]
  return header, teamA, teamB, mapasJuntos

In [10]:
def linkExtractor(url):
  """With the url from matches all in vlr, create a list of links with
  all the matches.
  """
  toGather = []
  page = urlopen(url)
  html = page.read().decode("utf-8")
  soup = BeautifulSoup(html, "html.parser")
  topLink = soup.find_all('a',class_='wf-module-item match-item mod-color mod-left mod-bg-after-striped_purple mod-first')
  bottomlink = soup.find_all('a',class_='wf-module-item match-item mod-color mod-left mod-bg-after-striped_purple')
  for link in topLink:
    toGather.append("https://www.vlr.gg"+link['href'])
  for link in bottomlink:
    toGather.append("https://www.vlr.gg"+link['href'])
  return toGather

In [16]:
def serialExtractor(url):
  page = urlopen(url)
  html = page.read().decode("utf-8")
  soup = BeautifulSoup(html, "html.parser")
  rows = {"team":[],
            "rival":[],
            "map":[],
            "own_pistol_tt":[],
            "own_pistol_ct":[],
            "own_score_tt":[],
            "rival_score_tt":[],
            "own_score_ct":[],
            "rival_score_ct":[]}
  header, teamA, teamB, mapasJuntos = runExtractor(soup)
  writeFunction(fileName,teamA,teamB)

  rowInfo = extraerScore(soup,mapasJuntos,rows)
  dictWritter(fileNameDict,rowInfo)
  time.sleep(random.randint(1, 3)) #Sleep de 1 a 3 sec para no floodear la pagina

In [18]:
urlLinks = "https://www.vlr.gg/event/matches/1924/champions-tour-2024-pacific-kickoff/?series_id=all"
toExtract = linkExtractor(urlLinks)

In [26]:
toExtract.remove("https://www.vlr.gg/297258/paper-rex-vs-gen-g-champions-tour-2024-pacific-kickoff-playoffs-gf")

In [27]:
toExtract

['https://www.vlr.gg/296735/t1-vs-bleed-champions-tour-2024-pacific-kickoff-opening-a',
 'https://www.vlr.gg/296730/zeta-division-vs-global-esports-champions-tour-2024-pacific-kickoff-opening-a',
 'https://www.vlr.gg/296747/gen-g-vs-paper-rex-champions-tour-2024-pacific-kickoff-winners-c',
 'https://www.vlr.gg/296733/detonation-focusme-vs-team-secret-champions-tour-2024-pacific-kickoff-elim-b',
 'https://www.vlr.gg/296749/gen-g-vs-rex-regum-qeon-champions-tour-2024-pacific-kickoff-decider-c',
 'https://www.vlr.gg/297254/team-secret-vs-gen-g-champions-tour-2024-pacific-kickoff-play-ins',
 'https://www.vlr.gg/297256/t1-vs-paper-rex-champions-tour-2024-pacific-kickoff-playoffs-sf',
 'https://www.vlr.gg/296745/gen-g-vs-rex-regum-qeon-champions-tour-2024-pacific-kickoff-opening-c',
 'https://www.vlr.gg/296731/team-secret-vs-talon-esports-champions-tour-2024-pacific-kickoff-opening-b',
 'https://www.vlr.gg/296736/drx-vs-detonation-focusme-champions-tour-2024-pacific-kickoff-opening-b',
 'htt

In [28]:
#Preparación del bloque para correr el for
fileName = 'picksBans.csv'
fileNameDict='StatsMaps.csv'
header = ["team","ban1","ban2","le ban1","le ban2","pick1","decider","order"]
rows = {"team":[],
           "rival":[],
           "map":[],
           "own_pistol_tt":[],
           "own_pistol_ct":[],
           "own_score_tt":[],
           "rival_score_tt":[],
           "own_score_ct":[],
           "rival_score_ct":[]}
createCsvPicks(fileName,header)
createCsvDict(fileNameDict,rows)

In [29]:
#Aca arranca el for (probablemente se pueda optimizar y no es necesario correr en bucle limpiar las rows y pasar al csv)
#Problemas: Cuando en la lista de links no estan todos los partidos jugados
#Problemas: bo5 rompe picks y bans y no salen todos los tats
for dire in toExtract:
  serialExtractor(dire)

['team', 'ban1', 'ban2', 'le ban1', 'le ban2', 'pick1', 'decider', 'order'] ['BLD', 'Ascent', 'Sunset', 'Icebox', 'Bind', 'Breeze', 'Split', 'A'] ['T1', 'Icebox', 'Bind', 'Ascent', 'Sunset', 'Lotus', 'Split', 'B'] ['Breeze', 'Lotus', 'Split']
{'team': ['T1', 'T1', 'T1'], 'rival': ['BLEED', 'BLEED', 'BLEED'], 'map': ['Breeze', 'Lotus', 'Split'], 'own_pistol_tt': [0, 0, 1], 'own_pistol_ct': [0, 0, 0], 'own_score_tt': ['6', '9', 0], 'rival_score_tt': ['6', '3', 0], 'own_score_ct': ['6', '4', 0], 'rival_score_ct': ['6', '3', 0]}
['team', 'ban1', 'ban2', 'le ban1', 'le ban2', 'pick1', 'decider', 'order'] ['GE', 'Lotus', 'Ascent', 'Breeze', 'Split', 'Bind', 'Sunset', 'A'] ['ZETA', 'Breeze', 'Split', 'Lotus', 'Ascent', 'Icebox', 'Sunset', 'B'] ['Bind', 'Icebox', 'Sunset']
{'team': ['ZETA DIVISION', 'ZETA DIVISION', 'ZETA DIVISION'], 'rival': ['Global Esports', 'Global Esports', 'Global Esports'], 'map': ['Bind', 'Icebox', 'Sunset'], 'own_pistol_tt': [0, 1, 0], 'own_pistol_ct': [1, 1, 0], 'own

In [ ]:
rows = {"team":[],
           "rival":[],
           "map":[],
           "own_pistol_tt":[],
           "own_pistol_ct":[],
           "own_score_tt":[],
           "rival_score_tt":[],
           "own_score_ct":[],
           "rival_score_ct":[]} #for the Dicc

In [ ]:
#Webscraping
"""
Test de funcionamiento
Try con una pagina y ver elementos estructurales.
Buscar escalablidad del proceso y evaluar funciones necesarias
"""

#https://www.vlr.gg/307572/mibr-vs-sentinels-champions-tour-2024-americas-kickoff-play-ins
#https://www.vlr.gg/303087/evil-geniuses-vs-loud-champions-tour-2024-americas-kickoff-sf

url = "https://www.vlr.gg/307572/mibr-vs-sentinels-champions-tour-2024-americas-kickoff-play-ins"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [ ]:
#Running the functions
teams,banA,banB,pickA,pickB,decider = picksBansFiltrado(soup)

header, teamA, teamB = picksBansCsv(teams,banA,banB,pickA,pickB,decider)

mapasJuntos = [pickA[0],pickB[0],decider[0]]
mapasJuntos

['Split', 'Ascent', 'Sunset']

In [ ]:
#Bloque para correr con el for
fileName = 'picksBans.csv'
fileNameDict='StatsMaps.csv'
header = ["team","ban1","ban2","le ban1","le ban2","pick1","decider","order"]

createCsvPicks(fileName,header)
createCsvDict(fileNameDict,rows)
#Aca arranca el for (probablemente se pueda optimizar y no es necesario correr en bucle limpiar las rows y pasar al csv)

['https://www.vlr.gg/295605/nrg-esports-vs-furia-champions-tour-2024-americas-kickoff-opening-a',
 'https://www.vlr.gg/295610/loud-vs-sentinels-champions-tour-2024-americas-kickoff-opening-b']

In [ ]:
for dire in restricto:
  url = dire
  page = urlopen(url)
  html = page.read().decode("utf-8")
  soup = BeautifulSoup(html, "html.parser")
  rows = {"team":[],
            "rival":[],
            "map":[],
            "own_pistol_tt":[],
            "own_pistol_ct":[],
            "own_score_tt":[],
            "rival_score_tt":[],
            "own_score_ct":[],
            "rival_score_ct":[]}

  header, teamA, teamB, mapasJuntos = runExtractor(soup)
  writeFunction(fileName,teamA,teamB)
  rowInfo = extraerScore(soup,mapasJuntos,rows)
  dictWritter(fileNameDict,rowInfo)
  time.sleep(random.randint(1, 3)) #Sleep de 1 a 3 sec para no floodear la pagina


['Breeze', 'Ascent', 'Split']
{'team': ['NRG Esports', 'NRG Esports', 'NRG Esports'], 'rival': ['FURIA', 'FURIA', 'FURIA'], 'map': ['Breeze', 'Ascent', 'Split'], 'own_pistol_tt': [0, 1, 0], 'own_pistol_ct': [0, 0, 0], 'own_score_tt': ['5', '6', 0], 'rival_score_tt': ['4', '2', 0], 'own_score_ct': ['8', '7', 0], 'rival_score_ct': ['6', '6', 0]}
['Split', 'Ascent', 'Sunset']
{'team': ['LOUD', 'LOUD', 'LOUD'], 'rival': ['Sentinels', 'Sentinels', 'Sentinels'], 'map': ['Split', 'Ascent', 'Sunset'], 'own_pistol_tt': [0, 1, 0], 'own_pistol_ct': [1, 1, 0], 'own_score_tt': ['4', '5', 0], 'rival_score_tt': ['3', '2', 0], 'own_score_ct': ['9', '8', 0], 'rival_score_ct': ['8', '7', 0]}


In [ ]:
rows = {"team":[],
          "rival":[],
          "map":[],
          "own_pistol_tt":[],
          "own_pistol_ct":[],
          "own_score_tt":[],
          "rival_score_tt":[],
          "own_score_ct":[],
          "rival_score_ct":[]}

rowInfo = extraerScore(soup,mapasJuntos,rows)

In [ ]:
rowInfo

{'team': ['NRG Esports', 'NRG Esports', 'NRG Esports'],
 'rival': ['FURIA', 'FURIA', 'FURIA'],
 'map': ['Breeze', 'Ascent', 'Split'],
 'own_pistol_tt': [0, 1, 0],
 'own_pistol_ct': [0, 0, 0],
 'own_score_tt': ['5', '6', 0],
 'rival_score_tt': ['4', '2', 0],
 'own_score_ct': ['8', '7', 0],
 'rival_score_ct': ['6', '6', 0]}

In [ ]:
rowInfo = extraerScore(soup,mapasJuntos,rows) #Crea el archivo


In [ ]:
rowInfo

{'team': ['MIBR', 'MIBR', 'MIBR'],
 'rival': ['Sentinels', 'Sentinels', 'Sentinels'],
 'map': ['Ascent', 'Icebox', 'Sunset'],
 'own_pistol_tt': [0, 1, 0],
 'own_pistol_ct': [1, 0, 0],
 'own_score_tt': ['2', '5', 0],
 'rival_score_tt': ['3', '6', 0],
 'own_score_ct': ['5', '1', 0],
 'rival_score_ct': ['10', '7', 0]}

In [ ]:
#Intento de extraccion de links generales
#url de los matches del evento, con all. Ejemplo: https://www.vlr.gg/event/matches/1923/champions-tour-2024-americas-kickoff/?series_id=all
url = "https://www.vlr.gg/event/matches/1923/champions-tour-2024-americas-kickoff/?series_id=all"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [ ]:
topLink = soup.find_all('a',class_='wf-module-item match-item mod-color mod-left mod-bg-after-striped_purple mod-first')
bottomlink = soup.find_all('a',class_='wf-module-item match-item mod-color mod-left mod-bg-after-striped_purple')

In [ ]:
toGather = []
for link in topLink:
  toGather.append("https://www.vlr.gg"+link['href'])

for link in bottomlink:
  toGather.append("https://www.vlr.gg"+link['href'])

In [ ]:
toGather

['https://www.vlr.gg/295605/nrg-esports-vs-furia-champions-tour-2024-americas-kickoff-opening-a',
 'https://www.vlr.gg/295610/loud-vs-sentinels-champions-tour-2024-americas-kickoff-opening-b',
 'https://www.vlr.gg/295615/kr-esports-vs-g2-esports-champions-tour-2024-americas-kickoff-opening-c',
 'https://www.vlr.gg/295608/furia-vs-mibr-champions-tour-2024-americas-kickoff-elim-a',
 'https://www.vlr.gg/295607/nrg-esports-vs-cloud9-champions-tour-2024-americas-kickoff-winners-a',
 'https://www.vlr.gg/295617/g2-esports-vs-evil-geniuses-champions-tour-2024-americas-kickoff-winners-c',
 'https://www.vlr.gg/295609/cloud9-vs-mibr-champions-tour-2024-americas-kickoff-decider-a',
 'https://www.vlr.gg/307572/mibr-vs-sentinels-champions-tour-2024-americas-kickoff-play-ins',
 'https://www.vlr.gg/303087/evil-geniuses-vs-loud-champions-tour-2024-americas-kickoff-sf',
 'https://www.vlr.gg/303086/loud-vs-sentinels-champions-tour-2024-americas-kickoff-gf',
 'https://www.vlr.gg/295606/cloud9-vs-mibr-cham

In [ ]:
picksBans = soup.find("div",{"class":"match-header-note"}).get_text() #Extraccion del texto
picksBans = picksBans.strip() #Remover los separados y espacios
picksBans = picksBans.split(";")

teams,banA,banB,pickA,pickB,decider = picksBansFiltrado(picksBans)

header, teamA, teamB = picksBansCsv(teams,banA,banB,pickA,pickB,decider)

mapasJuntos = [pickA[0],pickB[0],decider[0]]
mapasJuntos

['Ascent', 'Icebox', 'Sunset']

In [ ]:
writeFunction(fileName,header,teamA,teamB)

[1, 14, 27, 40, 53, 66, 79, 92, 105, 118]

In [ ]:
#Scores (creo que no lo voy a usar)
#
scoreLeft = soup.find("div",{"style":"margin-right: 12px;"}).get_text() #Extraccion del texto
scoreLeft = scoreLeft.strip() #Remover los separados y espacios
scoreLeft = scoreLeft.split(";")

scoreRight = soup.find("div",{"style":"margin-left: 8px;"}).get_text() #Extraccion del texto
scoreRight = scoreRight.strip() #Remover los separados y espacios
scoreRight = scoreRight.split(";")

print(scoreLeft,scoreRight)

['7'] ['13']


In [ ]:
pistolRounds(soup,3)

In [ ]:
rowInfo

{'team': ['LOUD', 'LOUD', 'LOUD'],
 'rival': ['Sentinels', 'Sentinels', 'Sentinels'],
 'map': ['Split', 'Ascent', 'Sunset'],
 'own_pistol_tt': [0],
 'own_pistol_ct': [0],
 'own_score_tt': ['4', '5', 0],
 'rival_score_tt': ['3', '2', 0],
 'own_score_ct': ['9', '8', 0],
 'rival_score_ct': ['8', '7', 0]}

In [ ]:
filenameStats = "mapStats.csv" #File name of the stats


In [ ]:
dictWritter(filenameStats,rowInfo)

In [ ]:
with open("test.csv", "a") as csvfile:

    # pass the csv file to csv.writer function.
    writer = csv.writer(csvfile)

    # pass the dictionary keys to writerow
    # function to frame the columns of the csv file
    writer.writerow(rowInfo.keys())

    # make use of writerows function to append
    # the remaining values to the corresponding
    # columns using zip function.
    writer.writerows(zip(*rowInfo.values()))

In [ ]:


with open(filenameStats, 'a') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = rows)
    writer.writeheader()
    writer.writerows([rowInfo])
    csvfile.close()



In [ ]:
def dictWritter(filename=filenameStats,rows=rowInfo):
      with open(filename, 'a') as csvfile:
        writer.writeheader()
        writer.writerows([rows])


In [ ]:
dictWritter(filenameStats,rowInfo)

ValueError: I/O operation on closed file.

In [ ]:
field_names = ["team","rival","own_score_tt","rival_score_tt","own_score_ct","rival_score_ct"]
with open('Names.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = field_names)
    writer.writeheader()
    writer.writerows([rowInfo])

ValueError: dict contains fields not in fieldnames: 'own_pistol_ct', 'own_pistol_tt', 'map'

In [ ]:
bloques_equipos = soup.find_all('div', class_='team')
for bloque in bloques_equipos:
  rowInfo = []
  try:
    (bloque.find("div",class_="team-name").text.strip())
  except:
    pass

Evil Geniuses
LOUD
Evil Geniuses
LOUD
Evil Geniuses
LOUD


In [ ]:
for count, bloque in enumerate(bloques_equipos):
  print(count)
  try:
    equipoActual = bloque.find("div",class_="team-name").text.strip()
    print(equipoActual)
    equipo = bloque.find("div",class_="team-name").text.strip()
    spans = bloque.find_all("span")
    for span in spans:
      if span["class"][0] == "mod-ct":
        print(span.text)
      print(span)
  except:
    pass

0
Evil Geniuses
9
<span class="mod-ct">9</span>
<span class="mod-t">4</span>
1
LOUD
<span class="mod-t">3</span>
4
<span class="mod-ct">4</span>
2
3
4
Evil Geniuses
4
<span class="mod-ct">4</span>
<span class="mod-t">3</span>
5
LOUD
<span class="mod-t">8</span>
5
<span class="mod-ct">5</span>
6
7
8
Evil Geniuses
<span class="mod-t">4</span>
1
<span class="mod-ct">1</span>
9
LOUD
8
<span class="mod-ct">8</span>
<span class="mod-t">5</span>
10
11


In [ ]:
print(spans[0]["class"][0])
print(spans[0].text)

mod-ct
8


In [ ]:
for bloque in bloques_equipos:
  #print(bloque)
  equipo = bloque.get_text().strip()
  print(equipo)
  spans = bloque.find_all("span")
  print(spans)

13 


							Evil Geniuses						
9 /
						4
[<span class="mod-ct">9</span>, <span class="mod-t">4</span>]
LOUD						
3 /
						4

7
[<span class="mod-t">3</span>, <span class="mod-ct">4</span>]
EG
[]
LOUD
[]
7 


							Evil Geniuses						
4 /
						3
[<span class="mod-ct">4</span>, <span class="mod-t">3</span>]
LOUD						
8 /
						5

13
[<span class="mod-t">8</span>, <span class="mod-ct">5</span>]
EG
[]
LOUD
[]
5 


							Evil Geniuses						
4 /
						1
[<span class="mod-t">4</span>, <span class="mod-ct">1</span>]
LOUD						
8 /
						5

13
[<span class="mod-ct">8</span>, <span class="mod-t">5</span>]
EG
[]
LOUD
[]


In [ ]:


  puntajes = [int(spans.text.strip()) for span in spans]

[]


In [ ]:
 print(spans)

[]


In [ ]:
equipo = soup.find('div', class_='team-name').text.strip()

# Obtener los puntajes
puntajes = soup.find_all('span')

# Convertir los puntajes a una lista de números
puntajes = [int(puntaje.text.strip()) for puntaje in puntajes]

print(f"Equipo: {equipo}")
print(f"Puntajes: {puntajes}")

ValueError: invalid literal for int() with base 10: 'BETA2'

In [ ]:
temp1 = ""
temp2 = ""
for count, element in enumerate(picksBans):
  if count == 0:
    temp1 = element.split()[0]


print(temp1)

LOUD


In [ ]:
print(soup.find(id="match-header-vs-score"))

None


In [ ]:
#Execepciones
class RequestsException(Exception):

    def __init__(self, message, code=500):
        super(RequestsException, self).__init__(message)
        self.code = code

In [ ]:
#import liquipediapy.exceptions as ex (Esta es la fila de arriba)
from bs4 import BeautifulSoup
import requests
from urllib.request import quote


class liquipediapy():
	def __init__(self, appname: str, game: str, debug_folder: str= ""):
		self.appname = appname
		self.game = game
		self.__headers = {'User-Agent': appname, 'Accept-Encoding': 'gzip'}
		self.__base_url = 'https://liquipedia.net/'+game+'/api.php?'

		# If enabled, offer the possibility to
		# write the output of parse.page_html to a folder
		# and reading from it afterwards to avoid surcharing liquipedia api
		# If read mode and file requested not in folder, query will happen
		"""self.__debug = False
		if debug_folder != "":
			self.devparser = DevParser(debug_folder)
			self.__debug = True"""

	def parse(self,page):
		success, soup = False, None

		# If page.html is readable from file
		if self.__debug and self.devparser.isPageAvailableLocally(page):
			success, soup = self.devparser.fromFile(page)
		else:
			url = self.__base_url+'action=parse&format=json&page='+page
			response = requests.get(url, headers=self.__headers)
			if response.status_code == 200:
				try:
					page_html = response.json()['parse']['text']['*']
				except KeyError:
					raise RequestsException(response.json(),response.status_code)
			else:
				raise RequestsException(response.json(),response.status_code)

			soup = BeautifulSoup(page_html,features="lxml")

		# If page.html wasn't readable from file, write it for next time
		if (self.__debug and not self.devparser.isPageAvailableLocally(page)):
			self.devparser.toFile(soup, page)

		redirect = soup.find('ul',class_="redirectText")
		if redirect is None:
			return soup,None
		else:
			redirect_value = soup.find('a').get_text()
			redirect_value = quote(redirect_value)
			soup,__ = self.parse(redirect_value)
			return soup,redirect_value



	def dota2webapi(self,matchId):
		if self.game == 'dota2':
			url = self.__base_url+'action=dota2webapi&data=picks_bans|players|kills_deaths|duration|radiant_win|teams|start_time&format=json&matchid='+matchId
			response = requests.get(url, headers=self.__headers)
			if response.status_code == 200:
				res = response.json()
				if res['dota2webapi']['isresult'] >= 1:
					return res['dota2webapi']['result']
				else:
					return res['dota2webapi']['result']['error']
			else:
				raise ex.RequestsException(response.json(),response.status_code)
		else:
			raise ex.RequestsException('set game as dota2 to access this api',0)



	def search(self,serachValue):
		url = self.__base_url+'action=opensearch&format=json&search='+serachValue
		response = requests.get(url, headers=self.__headers)
		if response.status_code == 200:
			return  response.json()
		else:
			raise RequestsException(response.json(),response.status_code)

In [ ]:
liquipy_object = liquipediapy('appname',"dota2")

In [ ]:
response = requests.get('https://liquipedia.net/valorant/api.php?')

In [ ]:
response.status_code

200

In [ ]:
print(response)

<Response [200]>
